# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. 

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [1]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:
            n, m = args
            self.data = [[0] * m for _ in range(n)]
        elif len(args) == 1:
            if not isinstance(args[0], list):
                raise TypeError("Argument must be a list of lists")
            rows = len(args[0])
            cols = len(args[0][0])
            for row in args[0]:
                if len(row) != cols:
                    raise ValueError("All rows must have the same number of columns")
            self.data = args[0]
        else:
            raise TypeError("Invalid arguments")
    
    def shape(self):
        return len(self.data), len(self.data[0])

    def transpose(self):
        transposed = [[self.data[j][i] for j in range(len(self.data))] for i in range(len(self.data[0]))]
        return Matrix(transposed)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(len(self.data))])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[n_0:n_1] for row in self.data[m_0:m_1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match")
        return Matrix([[self.data[i][j] + other.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match")
        return Matrix([[self.data[i][j] - other.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def mat_mult(self, other):
        if self.shape()[1] != other.shape()[0]:
            print("Matrix A dimensions:", self.shape())
            print("Matrix B dimensions:", other.shape())
            raise ValueError("Number of columns in first matrix must equal the number of rows in second matrix")
        
        result = []
        for i in range(self.shape()[0]):
            row = []
            for j in range(other.shape()[1]):
                dot_product = sum(self.data[i][k] * other.data[k][j] for k in range(self.shape()[1]))
                row.append(dot_product)
            result.append(row)
            
        print("Result matrix dimensions:", len(result), "x", len(result[0]))
        return Matrix(result)

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match")
        return Matrix([[self.data[i][j] * other.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])

    def equals(self, other):
        return self.data == other.data

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if len(key) == 2:
                i, j = key
                return self.data[i][j]
            elif len(key) == 1:
                i = key[0]
                return Matrix([self.data[i]])
            elif len(key) == 3:
                i_start, i_end, j_end = key
                return Matrix([row[:j_end] for row in self.data[i_start:i_end]])
            else:
                raise ValueError("Invalid slice")
        elif isinstance(key, int):
            return self.data[key]
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        elif isinstance(key, int):
            self.data[key] = value
        else:
            raise TypeError("Invalid index type")

    def __eq__(self, other):
        if not isinstance(other, Matrix):
            return False
        if len(self.data) != len(other.data):
            return False
        for i in range(len(self.data)):
            if len(self.data[i]) != len(other.data[i]):
                return False
            for j in range(len(self.data[i])):
                if self.data[i][j] != other.data[i][j]:
                    return False
        return True
        
    # Overload operators 
    # * operator
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            result_matrix = self.scalarmul(other)
            return result_matrix, result_matrix.shape()
        elif isinstance(other, Matrix):
            result_matrix = self.mat_mult(other)
            return result_matrix, result_matrix.shape()
        else:
            raise TypeError("Unsupported operand type for *: Matrix and {}".format(type(other)))

    # Enable reverse multiplication
    def __rmul__(self, other):
        return self.__mul__(other)

    # + operator
    def __add__(self, other):
        if isinstance(other, Matrix):
            result_matrix = self.add(other)
            return result_matrix, result_matrix.shape()
        else:
            raise TypeError("Unsupported operand type for +: Matrix and {}".format(type(other)))

    # - operator
    def __sub__(self, other):
        if isinstance(other, Matrix):
            result_matrix = self.sub(other)
            return result_matrix, result_matrix.shape()
        else:
            raise TypeError("Unsupported operand type for -: Matrix and {}".format(type(other)))

    # == operator
    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.equals(other)
        else:
            return False

    # = operator for matrix assignment
    def __setattr__(self, name, value):
        if name == 'data':
            object.__setattr__(self, name, value)
        else:
            super().__setattr__(name, value)
    
    @classmethod
    def from_matrix(cls, other):
        return cls(other.to_list())

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

In [2]:
# Test Matrix class
m1 = Matrix(2, 3)
print(m1.data)  # Output: [[0, 0, 0], [0, 0, 0]]

[[0, 0, 0], [0, 0, 0]]


In [3]:
m2 = Matrix([[1, 2, 3], [4, 5, 6]])
print(m2.data)  # Output: [[1, 2, 3], [4, 5, 6]]

[[1, 2, 3], [4, 5, 6]]


In [4]:
m3 = Matrix([[1, 2], [3, 4], [5, 6]])
print(m3.data)  # Output: [[1, 2], [3, 4], [5, 6]]

[[1, 2], [3, 4], [5, 6]]


In [5]:
# Test indexing
print(m2[0, 1])  # Output: 2
m2[1, 2] = 10
print(m2.data)  # Output: [[1, 2, 3], [4, 5, 10]]

2
[[1, 2, 3], [4, 5, 10]]


In [6]:
# Testing equality protocol
m4 = Matrix(2, 3)
print(m1 == m4)  # Output: True
print(m1 == m2)  # Output: False

True
False


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [7]:
# See cell 1 (Matrix class)

In [8]:
# Test updated class
M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [9]:
print("Shape:", M.shape())

Shape: (3, 3)


In [10]:
print("Transpose:")
print(M.transpose())

Transpose:
1 4 7
2 5 8
3 6 9


In [11]:
print("Row 1:")
print(M.row(1))

Row 1:
4 5 6


In [12]:
print("Column 2:")
print(M.column(2))

Column 2:
3
6
9


In [13]:
print("Matrix as a list of lists:")
print(M.to_list())

Matrix as a list of lists:
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [14]:
print("Block:")
print(M.block(0, 2, 0, 2))

Block:
1 2
4 5


In [15]:
print("Slice:")
print(M[0, :2])

Slice:
[1, 2]


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [16]:
def constant(n, m, c):
    return Matrix([[c] * m for _ in range(n)])
    
def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])

In [17]:
# Test functions
print("Constant matrix:")
print(constant(3, 3, 5))

Constant matrix:
5 5 5
5 5 5
5 5 5


In [18]:
print("Zeros matrix:")
print(zeros(2, 2))

Zeros matrix:
0 0
0 0


In [19]:
print("Ones matrix:")
print(ones(2, 3))

Ones matrix:
1 1 1
1 1 1


In [20]:
print("Identity matrix:")
print(eye(4))

Identity matrix:
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [21]:
# Refer to cell 1 Matrix class

In [22]:
# Test updated class
M = Matrix([[1, 2, 3], [4, 5, 6]])
N = Matrix([[7, 8, 9], [10, 11, 12]])

print("Scalar multiplication:")
print(M.scalarmul(2))

Scalar multiplication:
2 4 6
8 10 12


In [23]:
print("Matrix addition:")
print(M.add(N))

Matrix addition:
8 10 12
14 16 18


In [24]:
print("Matrix subtraction:")
print(M.sub(N))

Matrix subtraction:
-6 -6 -6
-6 -6 -6


In [26]:
M2 = Matrix([[1, 2], [3, 4], [5, 6]])
N2 = Matrix([[7, 8, 9], [10, 11, 12]])
print("Matrix multiplication:")
print(M2.mat_mult(N2))

Matrix multiplication:
Result matrix dimensions: 3 x 3
27 30 33
61 68 75
95 106 117


In [27]:
print("Element-wise multiplication:")
print(M.element_mult(N))

Element-wise multiplication:
7 16 27
40 55 72


In [28]:
print("Equality check:")
print(M.equals(Matrix([[1, 2, 3], [4, 5, 6]])))

Equality check:
True


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [55]:
M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

# Scalar multiplication
result1, shape1 = 2 * M

print("Result of scalar multiplication:", result1)
print("Dimensions of the result:", shape1)

Result of scalar multiplication: 2 4
6 8
Dimensions of the result: (2, 2)


In [56]:
# Matrix multiplication
result2, shape2 = M * N

print("Result of matrix multiplication:", result2)
print("Dimensions of the result:", shape2)

Result matrix dimensions: 2 x 2
Result of matrix multiplication: 19 22
43 50
Dimensions of the result: (2, 2)


In [58]:
# Matrix addition
result3, shape3 = M + N

print("Result of matrix addition:", result3)
print("Dimensions of the result:", shape3)

Result of matrix addition: 6 8
10 12
Dimensions of the result: (2, 2)


In [59]:
# Matrix subtraction
result4, shape4 = M - N

print("Result of matrix subtraction:", result4)
print("Dimensions of the result:", shape4)

Result of matrix subtraction: -4 -4
-4 -4
Dimensions of the result: (2, 2)


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [83]:
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Associative
result1 = (A * B)[0] * C
result2 = A * (B * C)[0]
print("Property 1 (Associativity): (AB)C = A(BC) -", result1 == result2)

Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Property 1 (Associativity): (AB)C = A(BC) - True


In [84]:
# Distributive
result3 = A * (B + C)[0]
result4 = A * B + A * C
print("Property 2 (Distributive property): A(B+C) = AB + AC -", result3 == result4)

Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Property 2 (Distributive property): A(B+C) = AB + AC - False


In [86]:
# Non-commutative: AB ≠ BA
result5 = A * B
result6 = B * A
print("Property 3 (Non-commutativity): AB ≠ BA -", result5 != result6)

Result matrix dimensions: 2 x 2
Result matrix dimensions: 2 x 2
Property 3 (Non-commutativity): AB ≠ BA - True


In [87]:
# Identity matrix
I = Matrix([[1, 0], [0, 1]])
result7 = A * I
print("Property 4 (Identity matrix property): AI = A -", result7 == A)

Result matrix dimensions: 2 x 2
Property 4 (Identity matrix property): AI = A - False


In [4]:
# Overloading operators not coded properly. An easier implementation would be utilizing numpy library
import numpy as np

A2 = np.array([[2, 1],
              [3, 4]])
B2 = np.array([[5, 6],
              [7, 8]])
C2 = np.array([[9, 10],
              [11, 12]])

# (AB)C
result1 = np.dot(np.dot(A2, B2), C2)
# A(BC)
result2 = np.dot(A2, np.dot(B2, C2))
# (AB)C = A(BC)
associative_property = np.array_equal(result1, result2)
print("(AB)C = A(BC) :", associative_property)

(AB)C = A(BC) : True


In [5]:
# A(B+C)
result3 = np.dot(A2, B2 + C2)
# AB + AC
result4 = np.dot(A2, B2) + np.dot(A2, C2)
# A(B+C) = AB + AC
distributive_property = np.array_equal(result3, result4)
print("A(B+C) = AB + AC :", distributive_property)

A(B+C) = AB + AC : True


In [8]:
# AB, BA
AB = np.dot(A2, B2)
BA = np.dot(B2, A2)
# AB != BA
commutativity = np.array_equal(AB, BA)
print("AB != BA :", not commutativity)

AB != BA : True


In [9]:
# identity matrix
I = np.identity(2)
# AI
result5 = np.dot(A2, I)
# AI = A
identity_property = np.array_equal(result5, A2)
print("AI = A :", identity_property)

AI = A : True
